符号式编程可以对计算过程进行大量的优化，从而加速计算。

TF2.0和Keras增加了命令式编程，但符号式编程和计算图仍可通过装饰器`tf.function`进行访问。

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import d2l

In [2]:
def get_net():
    net = tf.keras.Sequential()
    net.add(Dense(256, input_shape = (512,), activation = "relu"))
    net.add(Dense(128, activation = "relu"))
    net.add(Dense(2, activation = "linear"))
    return net

In [4]:
x = tf.random.normal([1,512])

In [6]:
net = get_net()
net(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.8696447 ,  0.01650664]], dtype=float32)>

In [7]:
net = tf.function(net)
net(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.8696447 ,  0.01650664]], dtype=float32)>

通过将`jit_compile = True`标志添加到`tf.function()`的函数调用中可以显式地启用TensorFlow中的XLA（线性代数加速）功能。在某些情况下，XLA可以进一步优化JIT的编译代码。如果没有这种显式定义，图形模式将会被启用，但是XLA可以使某些大规模的线性代数的运算速度更快（与我们在深度学习应用程序中看到的操作类似），特别是在GPU环境中。

In [8]:
class Benchmark:
    def __init__(self,description='Done'):
        self.description = description
    def __enter__(self):
        self.timer = d2l.Timer()
        return self
    def __exit__(self,*args):
        print(f'{self.description}:{self.timer.stop():.4f}sec')

In [12]:
net = get_net()
with Benchmark('Eager模式'):
    for i in range(1000): net(x)

net = tf.function(net)
with Benchmark('Graph模式'):
    for i in range(1000): net(x)
    
net = tf.function(net,jit_compile=True)
with Benchmark('XLA模式'):
    for i in range(1000): net(x)

Eager模式:0.5584sec
Graph模式:0.2118sec
XLA模式:0.1898sec


In [13]:
net = get_net()
tf.saved_model.save(net, 'my_mlp')
!ls -lh my_mlp*

INFO:tensorflow:Assets written to: my_mlp/assets
总用量 68K
drwxr-xr-x 2 agave agave 4.0K 1月  13 12:46 assets
-rw-rw-r-- 1 agave agave  59K 1月  13 12:46 saved_model.pb
drwxr-xr-x 2 agave agave 4.0K 1月  13 12:46 variables


2022-01-13 12:46:20.955258: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
